# Ejercicio 2

En el siguiente problema, se presenta un conjunto de datos correspondientes a escritos de Shakespear. El objetivo del problema es crear un modelo capaz de generar texto con dialecto de época y escritura en verso y prosa.

# Librerías y entorno

In [1]:
# Install tensorflow 2.15
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installatio

In [2]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=36496eecf6561f1a7b39d44f80ef8381c392dda4d30338d9c7d5411f962275be
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [3]:
import requests

import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer

from googletrans import Translator


In [4]:
translator = Translator()

In [5]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Dataset


In [6]:
# URL del dataset
url = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"

# Realizar la solicitud de descarga
response = requests.get(url)

# Guardar el archivo
with open("shakespeare.txt", "wb") as file:
    file.write(response.content)

print("Descarga completada.")


Descarga completada.


In [7]:
# Read, then decode for py2 compat.
text = open("shakespeare.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 2000 characters in text
print(text[:2000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


# Modelo carácter a carácter

## Preprocesamiento

### Vectorización

Previo al entrenamiento, necesitamos convertir el texto a una representacion numerica.

La capa tf.keras.layers.StringLookup nos permite convertir cada caracter en un ID numerico. Solo necesita que el texto este separado primero en tokens.

Ahora creamos la capa tf.keras.layers.StringLookup:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

Esto nos convierte de tokens a IDs de caracteres

Dado que el proposito de este ejercicio es generar texto, tambien sera importante invertir esta representacion y recuperar texto legible desde estos IDs. Para esto utilizamos `tf.keras.layers.StringLookup(..., invert=True).`

Nota: Aquí, en lugar de pasar el vocabulario original generado con `sorted(set(text))`, usamos el método `get_vocabulary()` de la capa `tf.keras.layers.StringLookup` para que los tokens `[UNK]` se configuren de la misma manera.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Esta capa recupera los caracteres desde los vectores de IDs y los retorna como un `tf.RaggedTensor` de caracteres:



Finalmente usando `tf.strings.reduce_join` se pueden volver a juntar los caracteres en texto.

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# Convertimos el texto en ids numéricos
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


## Predicción carácter a carácter


Dado un caracter, o una secuencia de caracteres, ¿cuál es el siguiente caracter más probable? Esta es la tarea para la que estamos entrenando al modelo. La entrada al modelo será una secuencia de caracteres y entrenamos el modelo para predecir la salida: el siguiente carácter en cada paso de tiempo.

Dado que los RNN mantienen un estado interno que depende de los elementos vistos anteriormente, a partir de todos los caracteres calculados hasta este momento, ¿cuál es el siguiente carácter?

## Crear los ejemplos de entrenamiento

Dividimos el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá `seq_length` caracteres del texto.

Para cada secuencia de entrada, los targets correspondientes contienen la misma longitud de texto, excepto que se desplazan un carácter hacia la derecha.

Así que divida el texto en fragmentos de `seq_length+1`. Por ejemplo, digamos `que seq_length` es 3 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia target "ola".

Para hacer esto, usamos la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres.

In [ ]:
seq_length = 100

El método `batch` nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.



In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


Es más fácil ver lo que esta haciendo si unimos de vuelta los tokens en texto:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para el entrenamiento, necesitaremos un conjunto de datos de pares `(input, label)`. Donde `input` y `label` son secuencias. En cada timestep, la entrada es el carácter actual y la etiqueta es el siguiente carácter.

Aquí hay una función que toma una secuencia como entrada, la duplica y la desplaza para alinear la entrada y la etiqueta para cada timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Batches de entrenamiento

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, es necesario mezclarlos y batchearlos.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construccion del modelo


Este modelo tiene tres capas:

* `tf.keras.layers.Embedding`: La capa de entrada. Una lookup table entrenable que asignará cada ID de carácter a un vector con dimensiones `embedding_dim`;
* `tf.keras.layers.GRU`: una capa recurrente GRU de tamaño units=rnn_units (también se puede usar una capa LSTM aquí).
* `tf.keras.layers.Dense`: La capa de salida, con salidas` vocab_size`. Genera un logit para cada carácter del vocabulario. Estas son las probabilidades de cada caracter según el modelo.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Por cada caracter el modelo calcula su embedding, corre la GRU un timestep con el embedding como entrada y aplica la capa densa para generar los logits prediciendo la probabilidades del siguiente caracter.

## Probar el modelo

Ejecutamos el modelo para ver que se comporta como se esperaba.

Primero verificamos la shape de salida:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de la secuencia de la entrada es 100, pero el modelo se puede ejecutar con entradas de cualquier longitud:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Para obtener predicciones reales del modelo, se deben tomar muestras de la distribución de salida para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de los caracteres.

Nota: Es importante tomar una muestra de esta distribución, ya que tomar el argmax de la distribución puede fácilmente hacer que el modelo se atasque en un bucle.

Tomando como ejemplo el primero del batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Esto nos da para cada timestep una predicción del siguiente índice de caracteres:



In [ ]:
sampled_indices


array([14, 32, 42, 29, 10,  2,  2,  6, 50, 22, 18,  4, 32, 45, 63,  5, 52,
       41, 37, 32, 62, 52, 57, 44, 15, 57, 61, 57, 61,  5, 20,  7, 15, 59,
       58, 59, 29, 12, 12, 43, 37, 41, 35, 13, 40, 33, 30, 16,  7, 58, 34,
       16, 37, 50, 58, 32,  7, 19,  9, 42, 48, 31, 60, 62, 20,  1, 12,  3,
       53, 47, 45, 34, 50, 11, 14,  1, 11, 33, 37, 41,  8, 48, 62, 23,  4,
       54, 28, 18, 55,  6, 31, 16, 19, 19, 29,  9, 40, 55, 52,  0])

## Entrenamiento del modelo

El problema puede tratarse como un problema de clasificación estándar. Dado el estado RNN anterior y la entrada en este timestep, predice la clase del siguiente carácter.



**Agregamos un optimizador y una funcion costo**

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que su modelo devuelve logits, necesita configurar el indicador `from_logits`.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1898994, shape=(), dtype=float32)


Un modelo recién inicializado no debería estar demasiado seguro de sí mismo, todos los logits de salida deberían tener magnitudes similares. Para confirmar esto, puede comprobar que la exponencial del costo medio es aproximadamente igual al tamaño del vocabulario. Una pérdida mucho mayor significa que el modelo está seguro de sus respuestas incorrectas y está mal inicializado:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()


66.01615

Compilamos el modelo con `tf.keras.Model.compile` indicando el optimizador y la funcion costo:



In [ ]:
model.compile(optimizer='adam', loss=loss)


**Checkpoints del modelo**

Usamos el callback `tf.keras.callbacks.ModelCheckpoint` para que se guarden checkpoints del modelo durante el entrenamiento.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Ejecucion del entrenamiento**

Primero entrenamos con 20 épocas, al no obtener buenos resultos, decimos utilizar 40.

En el entrenamiento con 20 épocas notamos que el texto no era coherente.

In [ ]:
EPOCHS = 40

In [ ]:
tf.config.run_functions_eagerly(True)


In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 10s 43ms/step - loss: 2.7180
Epoch 2/40
172/172 [==============================] - 9s 42ms/step - loss: 1.9870
Epoch 3/40
172/172 [==============================] - 8s 42ms/step - loss: 1.7037
Epoch 4/40
172/172 [==============================] - 8s 41ms/step - loss: 1.5431
Epoch 5/40
172/172 [==============================] - 9s 43ms/step - loss: 1.4449
Epoch 6/40
172/172 [==============================] - 8s 42ms/step - loss: 1.3771
Epoch 7/40
172/172 [==============================] - 8s 42ms/step - loss: 1.3238
Epoch 8/40
172/172 [==============================] - 8s 43ms/step - loss: 1.2794
Epoch 9/40
172/172 [==============================] - 8s 43ms/step - loss: 1.2378
Epoch 10/40
172/172 [==============================] - 8s 42ms/step - loss: 1.1978
Epoch 11/40
172/172 [==============================] - 9s 42ms/step - loss: 1.1571
Epoch 12/40
172/172 [==============================] - 9s 42ms/step - loss: 1.1152
Epoch 13/40


## Generacion de texto

La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle y realizar un seguimiento del estado interno del modelo a medida que lo ejecutamos.



Cada vez que llamamos al modelo, pasamos algún texto y un estado interno. El modelo devuelve una predicción para el siguiente caracter y su nuevo estado. Vuelva a pasar la predicción y el estado para continuar generando texto.


Lo siguiente hace una predicción de un solo paso:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Lo ejecutamos en un bucle para generar texto. Al observar el texto generado, vemos que el modelo sabe cuándo poner mayúsculas, hacer párrafos e imita un vocabulario de escritura similar al de Shakespear. Con el reducido número de épocas de entrenamiento, todavía no ha aprendido a formar frases coherentes, incluso primero probamos con 20 épocas y luego con 40 y notamos el mismo compartamiento.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['To be or not to be'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

To be or not to be the office, yet his sure
With such a disinised against me one
Of what leave the servants of my mind
That hath brought so much sectars by your grace.
Still, good my lord, let me entreat of her
our father wear by a house of Marcius.

FRIAR PHEYCUS:
He'll prove a ted-mer you.

BENVOLIO:
Tut, you say is that these stinks of traging tongue?

GLOUCESTER:
The gods grant that spare and fled to give him gentlemen,
The leaves and frankings on others,
Therefore this other fling is not the stroke
And this fash-bold's virtue. Will you go along?

POLIXENES:
Keep at me! 'she'er, madam:
Against what man thou had, do make the crown;
Which are unavoiding in his sight, lords,
Let him not die, tranio. I like your ladyship
To use it as a persetual rock,
Which let their person proson bid gamm from him were,
Tell him with Richmond's worthy days again, Rubles.

KING LEWIS XI:
Warwick, what are you? were there give the nort? their lates
should Hortensio.

POMPEY:
Why,
As all turns a deel ove

Para poder interpretar mejor los resultados obtenidos, lo traducimos al español. La idea es poder analizar si el texto generado tiene coherencia.

In [ ]:
# Traducción al español
generated_text = result[0].numpy().decode('utf-8')
translated_text = translator.translate(generated_text, src='en', dest='es').text
print("Texto en español:\n")
print(translated_text, '\n\n' + '_'*80)

Texto en español:

Ser o no ser la oficina, sin embargo, su seguro
Con tan desinyado contra mí uno
De lo que deja a los sirvientes de mi mente
Eso ha traído tantos sectares por su gracia.
Aún así, bien mi señor, déjame suplicarla
Nuestro padre usa junto a una casa de Marcius.

Fray Pheycus:
Él demostrará un Ted-Mer tú.

Benvolio:
Tut, ¿dices que estos apestan la lengua trragante?

Gloucester:
Los dioses conceden ese repuesto y huyeron para darle caballeros,
Las hojas y los franqueos en los demás,
Por lo tanto, esta otra aventura no es el accidente cerebrovascular
Y la virtud de este boquiabierto.¿Vas a ir?

Polixenes:
¡Sigue conmigo!'Ella es, señora:
Contra lo que tenías, haz la corona;
Que son inevitantes en su vista, Señores,
No lo muera, Tranio.Me gusta tu señoría
Para usarlo como una roca persetual,
Que dejó que su persona Proson Bid Gamm de él fuera,
Dígale con los días dignos de Richmond nuevamente, rublos.

Rey Lewis XI:
Warwick, ¿qué eres?¿Hubo dando al Nort?sus laces
debería h

Consideramos que el modelo capta bien la estética y el estilo de Shakespeare pero necesita mejoras para generar contenido más coherente y significativo.
Probablemente la longitud de secuencia utilizada durante el entrenamiento influye en la capacidad del modelo para mantener la coherencia.

A continuación probaremos con distantes temperaturas y logitudes de secuencia.

Si queremos que el modelo genere texto *más rápido*, lo más fácil que se puede hacer es generar el texto por batches. En el siguiente ejemplo, el modelo genera 5 resultados aproximadamente en el mismo tiempo que tomó generar 1 arriba.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['To be or not to be', 'To be or not to be', 'To be or not to be', 'To be or not to be', 'To be or not to be'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)


tf.Tensor(
[b"To be or not to be a luast: yet stands at night.\nGod save the king! both powerful partner rife,\nTwoughts thy over-lean; then let me have I see\nthyself, friends, Gremits three great ones.\n\nFirst Murderer:\nAnve me, one Moring, of all kings enforced\nTo gass a gentle and forswearing at\nSome shape and mortal steel, to want\nThe ugleress of hot only.\n\nLEONTES:\nThou dost, and joid, my faith, for a\nnot that such sorrow buy why now ere I could see\nBut by Angelo is an honour that must\nDid I lay that live so hard hither. By your regiment.\n\nDORCAS:\nMen of Gloucester's death, for my defe girly, call home,\nGod will I stay, the ship spoke of Lancaster.\nBut, stay my throne, and better end o'er-dry them;\nWho, all this while he is affairs male.\nCome away; commend me to London,\nDo thought withal, and in her good cause\nBut but a schoolmaster with horses' us, for east\nWill prove the function of my brother:\nAnd why she's care for King of Hercules; beholdst kind\nwishin

## Evaluación del modelo caracter a caracter

In [ ]:
# Configuración de temperaturas y longitudes
temperaturas = [0.5, 1.0, 1.5]  # Baja, estándar y alta
longitudes = [50, 100, 200]  # Fragmentos cortos, medianos y largos
num_fragments = 5  # Número de fragmentos por configuración

# Generar fragmentos
generated_fragments = []
for temp in temperaturas:
    for length in longitudes:
        print(f"\n--- Temperatura: {temp}, Longitud: {length} ---\n")

        # Actualizar temperatura del modelo
        one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=temp)

        # Generar fragmentos para esta configuración
        for i in range(num_fragments):
            states = None
            next_char = tf.constant(["To be or not to be"])  # Texto inicial
            result = [next_char]

            # Generar texto con longitud específica
            for _ in range(length):
                next_char, states = one_step_model.generate_one_step(next_char, states=states)
                result.append(next_char)

            # Convertir a cadena y guardar
            text_new = tf.strings.join(result).numpy()[0].decode('utf-8')
            generated_fragments.append((temp, length, text_new))
            print(f"Fragmento {i+1}:\n{text_new}\n")



--- Temperatura: 0.5, Longitud: 50 ---

Fragmento 1:
To be or not to be a suitor?

MERCUTIO:
Nay, one sit in hope.

GREMI

Fragmento 2:
To be or not to be a suitor to my fear,
And many an old man's enemie

Fragmento 3:
To be or not to be a suitor?

MERENIUS:
Hear me, people!

EDWARD:
Bu

Fragmento 4:
To be or not to be a popty to him.

KING HENRY VI:
Warwick, speak an

Fragmento 5:
To be or not to be a suitor?

MERCUTIO:
Nay, I'll conduct his cousin


--- Temperatura: 0.5, Longitud: 100 ---

Fragmento 1:
To be or not to be a king'ed out?

BUCKINGHAM:
My lord, I swear to thee say amen.
I had a Raughty house, thy father Yo

Fragmento 2:
To be or not to be a solencer sound;
The tendering of all the world I am not for you.

KATHARINA:
They be it so.

DUKE

Fragmento 3:
To be or not to be a poor knave in her toward Gelly;
And yet we should, unless the duty throughly,
With a dogry with t

Fragmento 4:
To be or not to be a pupil of report
him. For a letter and my soul!
Think what you might c

## Conclusiones modelo caracter a caracter

**Fragmentos más relevantes**



1.   Temperatura: 0.5, Longitud: 200
Fragmento:

*To be or not to be a suitor?  
MERCUTIO:  
Nay, I'll come what I say, sir. I know this careful  
man that want nothing for the posterns: these flesh  
May be possessed with good and take in part  
With peace thy wounds to the p*

2.   Temperatura: 0.5, Longitud: 100


*To be or not to be a suitor to my soul.  
Canst thou not speak? O toward the shame of mine?  
JULIET:  
It is: and, in good*

3. Temperatura: 1.0, Longitud: 200
Fragmento:

*To be or not to be? I'll take in the rock's death.  
Now shall we do, if King Edward's friends,  
But kills away: hence she is Warwick's  
sweeting twought out of his chamber upon,  
and not a proud here. Come away; let him ca*

4. Temperatura: 1.5, Longitud: 100
Fragmento:

*To be or not to bear my nanks. Dors are  
My friends what's lost I give my conscience,  
Which by his head upon your pilgr*

5. Temperatura: 1.5, Longitud: 200

Fragmento:

*To be or not to be a luave to-morrow,  
Making and well obing.  

KATHARINA:  
A jest and sheken disdains to Rome.  

HORTENSIO:  
Father, be a Month'd, madam; see what I have seven years  
He hath had, the very windows tongue.*


**Conclusiones:**


1. Temperatura

* Baja (0.5):
La coherencia es alta, pero los textos tienden a ser menos creativos. Las frases son más predecibles y estructuradas

* Media (1.0):
Logra un equilibrio entre coherencia y creatividad. Los textos generados conservan el estilo de Shakespeare mientras permiten cierta flexibilidad en la composición.

* Alta (1.5):
Se observan más creatividad y palabras inventadas. Disminuye la coherencia

2. Longitud
* Fragmentos cortos (50):
Mayor coherencia en frases individuales, pero menos desarrollo de ideas.

* Fragmentos medianos (100):
Mejor desarrollo de ides y coherencia.

* Fragmentos largos (200):
Más incoherentes en frases que tienen temperatura alta.

# Modelo palabra a palabra

## Dataset

In [8]:
# URL del dataset
url = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"

# Realizar la solicitud de descarga
response = requests.get(url)

# Guardar el archivo
with open("shakespeare.txt", "wb") as file:
    file.write(response.content)

print("Descarga completada.")

Descarga completada.


In [9]:
# Read, then decode for py2 compat.
text = open("shakespeare.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


## Vectorización

In [10]:
# Dividir el texto en palabras
words = text.split()

# Crear una capa para convertir palabras en IDs numéricos
ids_from_words = tf.keras.layers.StringLookup(
    vocabulary=list(set(words)), mask_token=None
)

# Crear una capa para convertir IDs numéricos de vuelta a palabras
words_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None
)

def text_from_ids(ids):
    return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

# Convertimos el texto en IDs numéricos a nivel de palabras
all_ids = ids_from_words(words)
print(f"Total de palabras únicas: {len(ids_from_words.get_vocabulary())}")

# Crear el dataset a nivel de palabras
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Mostrar algunas palabras y sus IDs
for ids in ids_dataset.take(10):
    print(words_from_ids(ids).numpy().decode('utf-8'))


Total de palabras únicas: 25671
First
Citizen:
Before
we
proceed
any
further,
hear
me
speak.


## Predicción

### Batches de entramiento

In [11]:
# Definir la longitud de las secuencias (en palabras)
seq_length = 20  # Por ejemplo, 20 palabras por secuencia

# Agrupar las palabras en secuencias de longitud fija
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

# Mostrar una secuencia de palabras como verificación
for seq in sequences.take(1):
    print("Secuencia completa:", text_from_ids(seq).numpy().decode('utf-8'))

# Función para dividir la secuencia en entrada (input) y objetivo (target)
def split_input_target(sequence):
    input_text = sequence[:-1]  # Todo menos la última palabra
    target_text = sequence[1:]  # Todo menos la primera palabra
    return input_text, target_text

# Crear el dataset de entrada y objetivo
dataset = sequences.map(split_input_target)

# Mostrar un ejemplo de entrada y objetivo
for input_example, target_example in dataset.take(1):
    print("Input  :", text_from_ids(input_example).numpy().decode('utf-8'))
    print("Target :", text_from_ids(target_example).numpy().decode('utf-8'))


Secuencia completa: First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You are all resolved rather to
Input  : First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You are all resolved rather
Target : Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You are all resolved rather to


## Entrenamos el modelo palabra a palabra

### Construcción del modelo

In [12]:
# Longitud del vocabulario basado en palabras
vocab_size_words = len(ids_from_words.get_vocabulary())

# Dimensión del embedding (puedes mantenerlo igual o ajustarlo si lo consideras necesario)
embedding_dim_words = 256

# Número de unidades en la RNN
rnn_units_words = 1024


In [13]:
# Definir el modelo palabra a palabra
class WordLevelModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


In [14]:
# Instanciar el modelo palabra a palabra
model_2 = WordLevelModel(
    vocab_size=vocab_size_words,
    embedding_dim=embedding_dim_words,
    rnn_units=rnn_units_words
)


In [15]:
# Resumen del modelo
model_2.build(input_shape=(None, None))  # Define el tamaño de entrada como (batch_size, seq_length)
model_2.summary()

Model: "word_level_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  6571776   
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  26312775  
                                                                 
Total params: 36822855 (140.47 MB)
Trainable params: 36822855 (140.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Probamos el modelo

In [17]:
# Preparar el dataset en lotes para garantizar que se pase correctamente al modelo
BATCH_SIZE = 64
dataset_batched = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Probar el modelo palabra a palabra
for input_example_batch, target_example_batch in dataset_batched.take(1):
    example_batch_predictions = model_2(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size_words)"
    )


(64, 20, 25671) # (batch_size, sequence_length, vocab_size_words)


In [18]:
# Probar el modelo palabra a palabra
for input_example_batch, target_example_batch in dataset_batched.take(1):
    example_batch_predictions = model_2(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size_words)"
    )


(64, 20, 25671) # (batch_size, sequence_length, vocab_size_words)


## Entrenamiento del modelo


In [19]:
# Configurar la pérdida para el modelo palabra a palabra
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [20]:
# Calcular la pérdida inicial para verificar que todo esté bien
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size_words)")
print("Mean loss:        ", example_batch_mean_loss)
print("Perplexity:       ", tf.exp(example_batch_mean_loss).numpy())

Prediction shape:  (64, 20, 25671)  # (batch_size, sequence_length, vocab_size_words)
Mean loss:         tf.Tensor(10.153151, shape=(), dtype=float32)
Perplexity:        25671.854


In [21]:
model_2.compile(optimizer='adam', loss=loss)


In [22]:
# Configurar la carpeta de checkpoints
checkpoint_dir = './training_checkpoints_word_level'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)


In [23]:
# Definir número de épocas para el entrenamiento
EPOCHS = 20

In [24]:
# Entrenar el modelo palabra a palabra
history = model_2.fit(
    dataset_batched,  # Usamos el dataset en lotes
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/20
150/150 [==============================] - 13s 75ms/step - loss: 8.5576
Epoch 2/20
150/150 [==============================] - 3s 22ms/step - loss: 7.5345
Epoch 3/20
150/150 [==============================] - 3s 23ms/step - loss: 7.1653
Epoch 4/20
150/150 [==============================] - 3s 23ms/step - loss: 6.8151
Epoch 5/20
150/150 [==============================] - 3s 22ms/step - loss: 6.5047
Epoch 6/20
150/150 [==============================] - 3s 23ms/step - loss: 6.1728
Epoch 7/20
150/150 [==============================] - 3s 22ms/step - loss: 5.8116
Epoch 8/20
150/150 [==============================] - 3s 22ms/step - loss: 5.4333
Epoch 9/20
150/150 [==============================] - 3s 23ms/step - loss: 5.0589
Epoch 10/20
150/150 [==============================] - 3s 22ms/step - loss: 4.6745
Epoch 11/20
150/150 [==============================] - 3s 22ms/step - loss: 4.2670
Epoch 12/20
150/150 [==============================] - 3s 22ms/step - loss: 3.9071
Epoch 13/20


## Generación de texto

In [25]:
class OneStepWord(tf.keras.Model):
    def __init__(self, model, words_from_ids, ids_from_words, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.words_from_ids = words_from_ids
        self.ids_from_words = ids_from_words

        # Crear una máscara para evitar generar "[UNK]"
        skip_ids = self.ids_from_words(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Colocar un -inf en cada índice prohibido
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Igualar la forma al vocabulario
            dense_shape=[len(ids_from_words.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convertir cadenas a IDs de palabras
        input_words = tf.strings.split(inputs)  # Dividir por palabras
        input_ids = self.ids_from_words(input_words).to_tensor()

        # Ejecutar el modelo
        # predicted_logits.shape es [batch, word, next_word_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)

        # Solo usar la última predicción
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature

        # Aplicar la máscara de predicción: prevenir que se genere "[UNK]"
        predicted_logits = predicted_logits + self.prediction_mask

        # Muestrear los logits de salida para generar IDs de palabras
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convertir de IDs de palabras a palabras
        predicted_words = self.words_from_ids(predicted_ids)

        # Retornar las palabras y el estado del modelo
        return predicted_words, states


In [28]:
# Crear una instancia del generador palabra a palabra
one_step_word = OneStepWord(model_2, words_from_ids, ids_from_words, temperature=1.0)

# Entrada inicial
start = tf.constant(["To be or not to be"])
states = None

# Generar texto palabra a palabra
generated_text = ["To be or not to be"]  # Iniciar con la frase semilla
for _ in range(50):  # Generar 50 palabras
    next_word, states = one_step_word.generate_one_step(start, states=states)
    generated_text.append(next_word.numpy()[0].decode('utf-8'))  # Decodificar la palabra
    start = next_word  # Usar la última palabra generada como entrada

# Unir las palabras generadas en una cadena completa
print(' '.join(generated_text))


To be or not to be ready to Padua and weary CURTIS: My I may still lay at the rancour and as dear brother; 'tis no sleep private ten master, my joy do through my sharp and being a words: For I have all thee, a biting greyhound To live. First Servant: Why thus, not to


In [31]:
# Traducción al español
generated_text_str = ' '.join(generated_text)  # Join the list elements into a single string
translated_text = translator.translate(generated_text_str, src='en', dest='es').text
print("Texto en español:\n")
print(translated_text, '\n\n' + '_'*80)

Texto en español:

Estar o no estar listo para Padua y Weary Curtis: mi aún puedo acostarme en el rencor y como querido hermano;'No hay sueño privado diez maestro, mi alegría lo hace a través de mi aguda y siendo una palabras: porque tengo todo, un galgo para vivir.Primer sirviente: ¿Por qué así, no 

________________________________________________________________________________


## Evaluamos el modelo

In [32]:
# Configuración de temperaturas y longitudes
temperaturas = [0.5, 1.0, 1.5]  # Baja, estándar y alta
longitudes = [10, 20, 50]  # Fragmentos cortos, medianos y largos (en palabras)
num_fragments = 5  # Número de fragmentos por configuración

# Generar fragmentos
generated_fragments = []
for temp in temperaturas:
    for length in longitudes:
        print(f"\n--- Temperatura: {temp}, Longitud: {length} ---\n")

        # Actualizar temperatura del modelo
        one_step_model_word = OneStepWord(model_2, words_from_ids, ids_from_words, temperature=temp)

        # Generar fragmentos para esta configuración
        for i in range(num_fragments):
            states = None
            next_word = tf.constant(["To be or not to be"])  # Texto inicial
            result = ["To be or not to be"]  # Iniciar con la frase inicial

            # Generar texto con longitud específica
            for _ in range(length):
                next_word, states = one_step_model_word.generate_one_step(next_word, states=states)
                result.append(next_word.numpy()[0].decode('utf-8'))  # Decodificar palabra

            # Convertir a cadena y guardar
            text_new = ' '.join(result)  # Combinar palabras en un texto completo
            generated_fragments.append((temp, length, text_new))
            print(f"Fragmento {i+1}:\n{text_new}\n")



--- Temperatura: 0.5, Longitud: 10 ---

Fragmento 1:
To be or not to be ingrate. TRANIO: 'Tis a man that I can seek my

Fragmento 2:
To be or not to be ingrate. HORTENSIO: Padua and yet you are not in, the

Fragmento 3:
To be or not to be ingrate. HORTENSIO: Padua and the first become that idle tears:

Fragmento 4:
To be or not to be ingrate. HORTENSIO: Padua and entreat you to the Tower, Her

Fragmento 5:
To be or not to be ingrate. TRANIO: 'Tis a man that I knew my hell,


--- Temperatura: 0.5, Longitud: 20 ---

Fragmento 1:
To be or not to be ingrate. HORTENSIO: Padua and not prove a country, and how the use of thy breath to have a forfeit of

Fragmento 2:
To be or not to be ingrate. PETRUCHIO: Hortensio, 'tis said, one that goes passing labour and the first complaint; the great man, Even to be

Fragmento 3:
To be or not to be ingrate. GREMIO: So long by the Tower, my mirth, my son; That ever do not be. LUCENTIO: Go, call them

Fragmento 4:
To be or not to be ingrate. HORTENSIO: Pe

## Conclusiones del modelo palabra a palabra


**Fragmentos más relevantes**


**Ejemplo 1:** Temperatura 0.5, Longitud 50
--
*To be or not to be rid of mine. HORTENSIO: Signior Baptista, is my good report to her my son And come to touch thy face? Their Buckingham, sir. ESCALUS: My lord, my lord. DUKE VINCENTIO: What, shall be my fortune and my fortune knows my father had a little man at the first head to...*

El fragmento muestra alta coherencia en la estructura, con nombres de personajes y una narrativa que podría pasar como un diálogo del texto original. Sin embargo, repite estructuras como "my fortune" y pierde dirección en frases largas.

---

**Ejemplo 2:** Temperatura 1.0, Longitud 20
---
*To be or not to be your brother's son, Was it is at heart. Sound, Alack, my poor craves a froward folks, all these mortal sun.*

Este fragmento es más diverso, con un uso interesante de términos como "Alack" y frases metafóricas ("my poor craves a froward folks"). Aunque algunas frases carecen de sentido completo, el tono y vocabulario son consistentes con el estilo.

---

**Ejemplo 3:** Temperatura 1.5, Longitud 50
--
*To be or not to be fine Command our foul looking From AUFIDIUS: Would earth we too: child, my every should, This estimation when once this opens their only revenge and fellest throat, if twice passing well, And bring him eyes and God's good wife Than long lies away, at me, dead once fasting, and be...*

A pesar de la temperatura alta, se capturan palabras que parecen del estilo ("AUFIDIUS", "fellest throat"), pero el fragmento pierde coherencia.

---


# Conclusiones generales

**Coherencia**
--
* **Caracter a Carácter:**

Mayor capacidad para generar frases con coherencia gramatical básica, ya que construye palabras a partir de caracteres y sigue patrones frecuentes aprendidos en el texto.
Pierde coherencia narrativa en fragmentos largos y temperaturas altas, generando palabras inexistentes.

---

* **Palabra a Palabra:**

Genera estructuras más coherentes a nivel narrativo, ya que utiliza palabras completas como base.
Mantiene mejor el contexto en fragmentos largos, especialmente en temperaturas bajas y medias.

---

**Conclusión:**
El modelo palabra a palabra sobresale en coherencia gracias al uso de palabras completas.



**Creatividad**
--
* **Caracter a Carácter:**
Mayor diversidad en temperaturas altas debido a la construcción libre de palabras. Sin embargo, esto puede llevar a incoherencias o invenciones
como "luave".

---

* **Palabra a Palabra:**
Más restringido a palabras existentes del vocabulario, lo que reduce la creatividad en temperaturas bajas. En temperaturas altas, muestra combinaciones más inusuales, pero parece evitar palabras inventadas.

----

**Conclusión:**
El modelo carácter a carácter genera más creatividad en términos de vocabulario, mientras que el modelo palabra a palabra es más conservador pero genera combinaciones interesantes en temperaturas altas.

**Impacto de la Temperatura**
--
* **Temperatura Baja (0.5):**

Ambos modelos generan textos más estructurados y repetitivos. Sin embargo, el modelo palabra a palabra mantiene mejor el estilo sin invenciones extrañas.

* **Temperatura Media (1.0):**

Ambos logran un equilibrio entre coherencia y creatividad. El modelo palabra a palabra genera textos narrativamente más ricos, mientras que el carácter a carácter conserva mejor el flujo rítmico.

* **Temperatura Alta (1.5):**

El modelo carácter a carácter pierde sentido rápidamente con palabras inventadas y narrativas desconectadas. El palabra a palabra conserva términos existentes, aunque las combinaciones son menos coherentes.

---

**Conclusión:**

 La temperatura media es ideal para ambos modelos, pero el palabra a palabra es más estable en temperaturas altas.

**Impacto de la Longitud**
--
* **Caracter a Carácter:**

**Fragmentos cortos (50 caracteres):**

Conserva coherencia en frases individuales.

**Fragmentos medianos (100 caracteres):**

Permite el desarrollo de ideas con cierto equilibrio.

**Fragmentos largos (200 caracteres):**

Pierde sentido rápidamente, especialmente en temperaturas altas.

---

**Palabra a Palabra:**
**Fragmentos cortos (10 palabras):**
Coherentes pero a veces demasiado simples.

**Fragmentos medianos (20 palabras):**

Logran el mejor equilibrio entre desarrollo narrativo y fluidez.

**Fragmentos largos (50 palabras):**

Mantienen el estilo, pero con un riesgo mayor de frases desconectadas en temperaturas altas.

----

**Conclusión:**
El modelo palabra a palabra gestiona mejor las narrativas en fragmentos largos.


**Estilo**
--
El modelo palabra a palabra no logra capturar la forma de poesía, el texto lo de vuelve como toda una oración, no devuelve los saltos de línea como si lo hace el modelo carácter a carácter.